# Title : Potential Retail Line Investment In Toronto and New York

## 1. Introduction

New York City and the city of Toronto are among the world busiest financial centres, and the vibrancies in these cities cater for vast amount of opportunities in setting up retail business. However, prior to putting in limited resources and investment, due diligence would be required to ensure that the highest chance of business success.

It would be appropriate to carry out the initial analysis using data as a platform to understand the differences and nature of retail business around the two locations. The primary questions would be as follows:

a. What would be the top 5 retail shops in both Toronto and New York business districts?

b. If someone wants to setup a retail line in Toronto or New York, which retail line would be a safe bet given the current cluster of shops in these cities?

Answering the above questions contribute towards the right direction subsequently during decision making to setup retail business. Here Python would be used as the data analysis platform to sieve out the required answers amongst data gather from various open sources.

## 2. Data Requirements

In order to identify the popular retail lines and enable higher probability of success when setting up new retail line within these cities, appropriate data would require to be gathered to perform the necessary analysis. The following data would be required to carry out analysis:

a. The list and type of retail shops in Toronto and New York business districts

b. The ratings and reviews on the type of popular retail lines in Toronto and New York

c. If trending data can be obtained, it will be an added advantage in determining the potential retail line that can be invested in.

## 3. Download and Explore Dataset

The Foursquare API is then used to explore neighbourhoods in the heart of New York (i.e. Manhattan) and Toronto. Foursquare API allows the application developers to interact with the Foursquare platform to obtain data related to a location such as retails shops around the neighbourhoods, comments on these shops and the popular venues around the locations. The explore function under Foursquare API is used to get the most common venue categories in each neighbourhood, and then use this feature to group the neighbourhoods into clusters using the k-means clustering algorithm.

With both of the dataset ready, Foursquare functions are then invoked to get information on the top 100 venues within 500m radius from each of the respective neighbourhoods from Manhattan and Toronto obtained and processed in earlier steps. 

Finally, the Folium library is used to visualize the neighbourhoods in New York City and their emerging clusters.

### a. Import Relevant Libraries

In [1]:
# library to handle data in a vectorized manner
import numpy as np 

# library for data analsysis
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# library to handle JSON files
import json 

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# library to handle requests
import requests 

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium 

print('Libraries imported.')

Libraries imported.


### b. Load and Preparing Data

#### --- --- ---   New York (Manhatten) Dataset   --- --- ---

In [2]:
# Download dataset for New York City
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Network - Data downloaded!')

Network - Data downloaded!


'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
# Extract data on Manhatten
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


#### --- --- ---   Toranto Dataset   --- --- ---

In [7]:
# Download dataset for Toronto City
table = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

df_table = table[0]
df_table.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
# Clean up the imported data
df_table.drop(df_table[df_table['Borough']=='Not assigned'].index, inplace=True)
df_table.reset_index(inplace=True)
df_table.drop('index', axis=1, inplace=True)

df_table["Neighborhood"]= df_table["Neighborhood"].str.split("/") 
df_table["Neighborhood"] = df_table["Neighborhood"].str.join(",") 
df_table.groupby(['Postal Code'], as_index = False).agg({'Neighborhood': ', '.join})

df_table['Neighborhood'] = df_table[['Neighborhood']].fillna('No value')
for x in range(df_table.shape[0]):
    if(df_table['Neighborhood'][x]=='No value'): #  np.nan):
        df_table['Neighborhood'][x]=df_table['Borough'][x]

df_table.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
geospatial = pd.read_csv("http://cocl.us/Geospatial_data")

df_newtable = pd.merge(df_table, geospatial, on='Postal Code')
df_newtable.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [10]:
toronto_data = df_newtable[df_newtable['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.drop('Postal Code', axis=1, inplace=True)

toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,East Toronto,The Beaches,43.676357,-79.293031


## 3. Exploratory Data Analysis

### a. Get Data from Foursquare

In [11]:
CLIENT_ID = 'NCZLWOXT14BXXS4YTDRZNUYZXXEUNN1EEINSXXZ05YJYXX1L' # your Foursquare ID - Masked
CLIENT_SECRET = 'TMITMDXXOAWYXXTQWU052XIGEDXLDGDTXXUHUQPNXXTP1XD5' # your Foursquare Secret - Masked
VERSION = '20180605' # Foursquare API version

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Get Manhattan Data From Foursquare

In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# Get info on venues
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [16]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3126, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


####    Get Toronto Data From Foursquare

In [17]:
# Get info on venues
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [18]:
print(toronto_venues.shape)
toronto_venues.head()

(1618, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


## 4.  Predictive Modelling

### a. One Hot Encoding for Manhattan

In [19]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restau

In [20]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,65,65,65,65,65,65
Carnegie Hill,86,86,86,86,86,86
Central Harlem,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,40,40,40,40,40,40
East Village,100,100,100,100,100,100


In [21]:
#manhattan_onehot.shape
manhattan_grouped0 = manhattan_onehot.groupby('Neighborhood').mean()
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restau

#### List of Top 5 hotspot in Manhattan

In [22]:
manhatten_hotspot=manhattan_grouped.sum(axis=0, skipna=True)
manhatten_hotspot.drop(manhatten_hotspot.index[0], inplace=True)
manhatten_hotspot.sort_values(ascending=False, inplace=True)
manhatten_hotspot.head(5) # Top 5 hotspot in Manhatten

Coffee Shop           1.82774
Italian Restaurant    1.29603
Park                  1.09857
Pizza Place           1.08864
Café                  1.08035
dtype: object

In [23]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.12
1          Hotel  0.06
2            Gym  0.05
3  Memorial Site  0.05
4    Coffee Shop  0.05


----Carnegie Hill----
                venue  freq
0         Coffee Shop  0.09
1         Pizza Place  0.05
2                Café  0.05
3  Italian Restaurant  0.03
4                 Gym  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.07
1                   Bar  0.04
2   American Restaurant  0.04
3  Gym / Fitness Center  0.04
4    Seafood Restaurant  0.04


----Chelsea----
            venue  freq
0     Coffee Shop  0.08
1     Art Gallery  0.08
2            Café  0.04
3  Ice Cream Shop  0.04
4          Bakery  0.03


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.06
1        Cocktail Bar  0.04
2              Bakery  0.04
3     Bubble Tea Shop  0.03
4                 Bar  0.03


----Civic Center----
          venue  freq
0   Coffee Shop  0.07
1         Hotel  0.05


In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Memorial Site,Gym,Coffee Shop,Plaza,Shopping Mall,Food Court,Gourmet Shop,Playground
1,Carnegie Hill,Coffee Shop,Pizza Place,Café,Bookstore,Gym / Fitness Center,Italian Restaurant,Gym,Yoga Studio,Wine Shop,Vietnamese Restaurant
2,Central Harlem,African Restaurant,Gym / Fitness Center,American Restaurant,French Restaurant,Chinese Restaurant,Bar,Seafood Restaurant,Food Truck,Gym,Library
3,Chelsea,Art Gallery,Coffee Shop,Café,Ice Cream Shop,American Restaurant,Bakery,Nightclub,Theater,Pizza Place,Cocktail Bar
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Bubble Tea Shop,Ice Cream Shop,Spa,Bar,Coffee Shop,Optical Shop,American Restaurant


#### Perform Clustering of Venues

In [26]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([0, 2, 2, 1, 2, 1, 1, 3, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 2, 1])

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Pharmacy,Supplement Shop,Steakhouse,Seafood Restaurant,Pizza Place,Deli / Bodega
1,Manhattan,Chinatown,40.715618,-73.994279,2,Chinese Restaurant,Bakery,Cocktail Bar,Bubble Tea Shop,Ice Cream Shop,Spa,Bar,Coffee Shop,Optical Shop,American Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Bakery,Spanish Restaurant,Deli / Bodega,Mobile Phone Shop,Coffee Shop,Latin American Restaurant,Supermarket,Sandwich Place,Tapas Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,3,Lounge,Mexican Restaurant,Restaurant,Bakery,Café,Frozen Yogurt Shop,Spanish Restaurant,Caribbean Restaurant,Chinese Restaurant,Park
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Pizza Place,Deli / Bodega,Coffee Shop,Café,Mexican Restaurant,Sushi Restaurant,Caribbean Restaurant,School,Chinese Restaurant,Sandwich Place


In [28]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


#### Plot Map of Manhattan with Clusters

In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Manhattan (Cluster 0)

In [30]:
manhattan_cluster0 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
manhattan_cluster0

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Roosevelt Island,Park,Soccer Field,Gym / Fitness Center,School,Scenic Lookout,Sandwich Place,Dry Cleaner,Liquor Store,Dog Run,Monument / Landmark
28,Battery Park City,Park,Hotel,Memorial Site,Gym,Coffee Shop,Plaza,Shopping Mall,Food Court,Gourmet Shop,Playground
37,Stuyvesant Town,Playground,Park,Heliport,Fountain,Farmers Market,Skating Rink,Boat or Ferry,Gas Station,Bistro,Bar


#### List of Top 5 hotspot in Manhattan (Cluster 0)

In [31]:
manhattan_cluster00 = manhattan_grouped0.loc[manhattan_cluster0.Neighborhood]
hotspot0=manhattan_cluster00.sum(axis=0, skipna=True)
hotspot0.drop(hotspot0.index[0], inplace=True)
hotspot0.sort_values(ascending=False, inplace=True)
hotspot0.head(5) # Top 5 hotspot in Manhattan Cluster 0

Park              0.303154
Playground        0.176363
Coffee Shop       0.136192
Farmers Market    0.090038
Baseball Field    0.090038
dtype: float64

### Manhattan (Cluster 1)

In [32]:
manhattan_cluster1 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
manhattan_cluster1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,Pizza Place,Deli / Bodega,Coffee Shop,Café,Mexican Restaurant,Sushi Restaurant,Caribbean Restaurant,School,Chinese Restaurant,Sandwich Place
14,Clinton,Theater,Gym / Fitness Center,Coffee Shop,Italian Restaurant,Hotel,Wine Shop,Gym,Pizza Place,American Restaurant,Spa
15,Midtown,Hotel,Coffee Shop,Bakery,Theater,Cuban Restaurant,Sushi Restaurant,Clothing Store,Japanese Restaurant,Pizza Place,Steakhouse
16,Murray Hill,Hotel,Sandwich Place,Pizza Place,Coffee Shop,Gym / Fitness Center,Japanese Restaurant,Cuban Restaurant,Deli / Bodega,Bar,Mediterranean Restaurant
17,Chelsea,Art Gallery,Coffee Shop,Café,Ice Cream Shop,American Restaurant,Bakery,Nightclub,Theater,Pizza Place,Cocktail Bar
20,Lower East Side,Chinese Restaurant,Art Gallery,Café,Bakery,Ramen Restaurant,Cocktail Bar,Nightclub,Juice Bar,Convenience Store,Latin American Restaurant
26,Morningside Heights,Park,American Restaurant,Bookstore,Coffee Shop,Sandwich Place,Deli / Bodega,Burger Joint,Café,Tennis Court,Outdoor Sculpture
32,Civic Center,Coffee Shop,Hotel,Cocktail Bar,American Restaurant,French Restaurant,Spa,Park,Café,Gym / Fitness Center,Yoga Studio
33,Midtown South,Korean Restaurant,Hotel,Dessert Shop,Café,Japanese Restaurant,Hotel Bar,Gym / Fitness Center,American Restaurant,Burger Joint,Coffee Shop


#### List of Top 5 hotspot in Manhattan (Cluster 1)

In [33]:
manhattan_cluster01 = manhattan_grouped0.loc[manhattan_cluster1.Neighborhood]
hotspot1=manhattan_cluster01.sum(axis=0, skipna=True)
hotspot1.drop(hotspot1.index[0], inplace=True)
hotspot1.sort_values(ascending=False, inplace=True)
hotspot1.head(5) # Top 5 hotspot in Manhattan Cluster 1

Coffee Shop            0.488645
Hotel                  0.302289
Café                   0.280763
Pizza Place            0.271978
American Restaurant    0.269930
dtype: float64

### Manhattan (Cluster 2)

In [34]:
manhattan_cluster2 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
manhattan_cluster2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Bubble Tea Shop,Ice Cream Shop,Spa,Bar,Coffee Shop,Optical Shop,American Restaurant
5,Manhattanville,Coffee Shop,Seafood Restaurant,Mexican Restaurant,Italian Restaurant,Park,Chinese Restaurant,Deli / Bodega,Bike Trail,American Restaurant,Supermarket
6,Central Harlem,African Restaurant,Gym / Fitness Center,American Restaurant,French Restaurant,Chinese Restaurant,Bar,Seafood Restaurant,Food Truck,Gym,Library
8,Upper East Side,Italian Restaurant,Coffee Shop,Gym / Fitness Center,Bakery,Yoga Studio,Spa,French Restaurant,American Restaurant,Juice Bar,Salad Place
9,Yorkville,Coffee Shop,Italian Restaurant,Gym,Bar,Pizza Place,Deli / Bodega,Sushi Restaurant,Mexican Restaurant,Japanese Restaurant,Wine Shop
10,Lenox Hill,Coffee Shop,Italian Restaurant,Pizza Place,Cocktail Bar,Café,Sushi Restaurant,Gym / Fitness Center,Gym,Burger Joint,Turkish Restaurant
12,Upper West Side,Italian Restaurant,Bar,Indian Restaurant,Wine Bar,Dessert Shop,Thai Restaurant,Coffee Shop,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant
13,Lincoln Square,Plaza,Italian Restaurant,Café,Gym / Fitness Center,Concert Hall,Performing Arts Venue,Theater,Wine Shop,Indie Movie Theater,American Restaurant
18,Greenwich Village,Italian Restaurant,Café,Sushi Restaurant,Gym,Burger Joint,Sandwich Place,Pizza Place,Bakery,Clothing Store,Chinese Restaurant
19,East Village,Bar,Mexican Restaurant,Cocktail Bar,Coffee Shop,Speakeasy,Wine Bar,Juice Bar,Pizza Place,Ice Cream Shop,Greek Restaurant


#### List of Top 5 hotspot in Manhattan (Cluster 2)

In [35]:
manhattan_cluster02 = manhattan_grouped0.loc[manhattan_cluster2.Neighborhood]
hotspot2=manhattan_cluster02.sum(axis=0, skipna=True)
hotspot2.drop(hotspot2.index[0], inplace=True)
hotspot2.sort_values(ascending=False, inplace=True)
hotspot2.head(5) # Top 5 hotspot in Manhattan Cluster 2

Italian Restaurant    1.116398
Coffee Shop           1.069296
Pizza Place           0.603982
Café                  0.598475
Bar                   0.548607
dtype: float64

### Manhattan (Cluster 3)

In [36]:
manhattan_cluster3 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
manhattan_cluster3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Bakery,Spanish Restaurant,Deli / Bodega,Mobile Phone Shop,Coffee Shop,Latin American Restaurant,Supermarket,Sandwich Place,Tapas Restaurant
3,Inwood,Lounge,Mexican Restaurant,Restaurant,Bakery,Café,Frozen Yogurt Shop,Spanish Restaurant,Caribbean Restaurant,Chinese Restaurant,Park
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Latin American Restaurant,Deli / Bodega,Taco Place,French Restaurant,Liquor Store,Beer Bar,Grocery Store
36,Tudor City,Park,Café,Mexican Restaurant,Deli / Bodega,Sushi Restaurant,Pizza Place,Gym / Fitness Center,Garden,Thai Restaurant,Dog Run


#### List of Top 5 hotspot in Manhattan (Cluster 3)

In [37]:
manhattan_cluster03 = manhattan_grouped0.loc[manhattan_cluster3.Neighborhood]
hotspot3=manhattan_cluster03.sum(axis=0, skipna=True)
hotspot3.drop(hotspot3.index[0], inplace=True)
hotspot3.sort_values(ascending=False, inplace=True)
hotspot3.head(5) # Top 5 hotspot in Manhattan Cluster 3

Mexican Restaurant    0.271236
Café                  0.201116
Bakery                0.198086
Deli / Bodega         0.159179
Park                  0.149482
dtype: float64

### Manhattan (Cluster 4)

In [38]:
manhattan_cluster4 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
manhattan_cluster4

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Pharmacy,Supplement Shop,Steakhouse,Seafood Restaurant,Pizza Place,Deli / Bodega


#### List of Top 5 hotspot in Manhattan (Cluster 4)

In [39]:
manhattan_cluster04 = manhattan_grouped0.loc[manhattan_cluster4.Neighborhood]
hotspot4=manhattan_cluster04.sum(axis=0, skipna=True)
hotspot4.drop(hotspot4.index[0], inplace=True)
hotspot4.sort_values(ascending=False, inplace=True)
hotspot4.head(5) # Top 5 hotspot in Manhattan Cluster 4

Sandwich Place     0.12
Coffee Shop        0.08
Gym                0.08
Yoga Studio        0.04
Supplement Shop    0.04
dtype: float64

### b. One Hot Encoding for Toronto

In [40]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot.drop('Neighborhood', axis=1, inplace=True)
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",

In [41]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,32,32,32,32,32,32


In [44]:
toronto_grouped0 = toronto_onehot.groupby('Neighborhood').mean() #NEW=================
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.018182,0.036364,0.000000,0.000000,0.00,0.018182,0.018182,0.000000,0.036364,0.000000,0.000000,0.018182,0.000000,0.0000,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036364,0.000000,0.0000,0.000000,0.000000,0.036364,0.000000,0.000000,0.000000,0.000000,0.018182,0.054545,0.072727,0.000000,0.00000,0.00000,0.000000,0.00,0.000000,0.0

#### List of Top 5 hotspot in Toronto

In [45]:
toronto_hotspot=toronto_grouped.sum(axis=0, skipna=True)
toronto_hotspot.drop(toronto_hotspot.index[0], inplace=True)
toronto_hotspot.sort_values(ascending=False, inplace=True)
toronto_hotspot.head(5) # Top 5 hotspot in Toronto

Coffee Shop            2.60855
Park                    2.0717
Café                   1.83484
Restaurant             1.09402
Italian Restaurant    0.871208
dtype: object

In [46]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2      Beer Bar  0.04
3          Café  0.04
4    Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1  Breakfast Spot  0.08
2     Coffee Shop  0.08
3       Nightclub  0.08
4          Bakery  0.08


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.13
1          Restaurant  0.07
2      Farmers Market  0.07
3             Brewery  0.07
4       Auto Workshop  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4           Airport  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1   Italian Restaurant  0.

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Bakery,Seafood Restaurant,Café,Restaurant,Clothing Store,Diner
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Breakfast Spot,Coffee Shop,Bakery,Performing Arts Venue,Pet Store,Convenience Store,Burrito Place,Stadium
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Park,Garden Center,Farmers Market,Fast Food Restaurant,Skate Park,Burrito Place,Butcher,Brewery,Auto Workshop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry,Plane
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Department Store,Salad Place,Burger Joint,Ramen Restaurant


#### Perform Clustering of Venues

In [48]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 2, 0])

In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Café,Theater,Beer Store,Ice Cream Shop,Chocolate Shop
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Mexican Restaurant,Diner,Bar,Bank,Beer Bar,Distribution Center
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Italian Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Café,Hotel,Tea Room
3,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Restaurant,Clothing Store,Gym,Creperie,Moroccan Restaurant
4,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Trail,Health Food Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio


#### Plot Map of Toronto with Clusters

In [50]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### c. List of Toronto Clusters & Most Common Venues

### Toronto (Cluster 0)

In [52]:
toronto_cluster0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
toronto_cluster0

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Café,Theater,Beer Store,Ice Cream Shop,Chocolate Shop
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Mexican Restaurant,Diner,Bar,Bank,Beer Bar,Distribution Center
2,"Garden District, Ryerson",Clothing Store,Coffee Shop,Italian Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Café,Hotel,Tea Room
3,St. James Town,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Restaurant,Clothing Store,Gym,Creperie,Moroccan Restaurant
4,The Beaches,Pub,Trail,Health Food Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
5,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Bakery,Seafood Restaurant,Café,Restaurant,Clothing Store,Diner
6,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Department Store,Salad Place,Burger Joint,Ramen Restaurant
7,Christie,Grocery Store,Café,Park,Restaurant,Baby Store,Italian Restaurant,Diner,Athletics & Sports,Nightclub,Coffee Shop
8,"Richmond, Adelaide, King",Coffee Shop,Café,Restaurant,Thai Restaurant,Hotel,Clothing Store,Deli / Bodega,Gym,Cosmetics Shop,Steakhouse
9,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Grocery Store,Pet Store,Music Venue,Middle Eastern Restaurant,Café,Bus Stop,Brewery,Supermarket


#### List of Top 5 hotspot in Toronto (Cluster 0)

In [53]:
toronto_cluster00 = toronto_grouped0.loc[toronto_cluster0.Neighborhood]
hotspot00=toronto_cluster00.sum(axis=0, skipna=True)
hotspot00.drop(hotspot00.index[0], inplace=True)
hotspot00.sort_values(ascending=False, inplace=True)
hotspot00.head(5) # Top 5 hotspot in Toronto Cluster 0

Coffee Shop           2.608555
Café                  1.834842
Park                  1.238369
Restaurant            1.094023
Italian Restaurant    0.871208
dtype: float64

### Toronto (Cluster 1)

In [54]:
toronto_cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
toronto_cluster1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Rosedale,Park,Playground,Trail,Yoga Studio,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run


#### List of Top 5 hotspot in Toronto (Cluster 1)

In [55]:
toronto_cluster01 = toronto_grouped0.loc[toronto_cluster1.Neighborhood]
hotspot01=toronto_cluster01.sum(axis=0, skipna=True)
hotspot01.drop(hotspot01.index[0], inplace=True)
hotspot01.sort_values(ascending=False, inplace=True)
hotspot01.head(5) # Top 5 hotspot in Toronto Cluster 1

Park                0.50
Playground          0.25
Trail               0.25
Department Store    0.00
Event Space         0.00
dtype: float64

### Toronto (Cluster 2)

In [56]:
toronto_cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
toronto_cluster2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### List of Top 5 hotspot in Toronto (Cluster 2)

In [57]:
toronto_cluster02 = toronto_grouped0.loc[toronto_cluster2.Neighborhood]
hotspot02=toronto_cluster02.sum(axis=0, skipna=True)
hotspot02.drop(hotspot02.index[0], inplace=True)
hotspot02.sort_values(ascending=False, inplace=True)
hotspot02.head(5) # Top 5 hotspot in Toronto Cluster 2

Park           0.333333
Swim School    0.333333
Bus Line       0.333333
Yoga Studio    0.000000
Diner          0.000000
dtype: float64

### Toronto (Cluster 3)

In [58]:
toronto_cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
toronto_cluster3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,"Forest Hill North & West, Forest Hill Road Park",Mexican Restaurant,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### List of Top 5 hotspot in Toronto (Cluster 3)

In [59]:
toronto_cluster03 = toronto_grouped0.loc[toronto_cluster3.Neighborhood]
hotspot03=toronto_cluster03.sum(axis=0, skipna=True)
hotspot03.drop(hotspot03.index[0], inplace=True)
hotspot03.sort_values(ascending=False, inplace=True)
hotspot03.head(5) # Top 5 hotspot in Toronto Cluster 3

Mexican Restaurant    0.25
Jewelry Store         0.25
Trail                 0.25
Sushi Restaurant      0.25
Yoga Studio           0.00
dtype: float64

### Toronto (Cluster 4)

In [60]:
toronto_cluster4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
toronto_cluster4

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Music Venue,Garden,Ice Cream Shop,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio,Department Store


#### List of Top 5 hotspot in Toronto (Cluster 4)

In [61]:
toronto_cluster04 = toronto_grouped0.loc[toronto_cluster4.Neighborhood]
hotspot04=toronto_cluster04.sum(axis=0, skipna=True)
hotspot04.drop(hotspot03.index[0], inplace=True)
hotspot04.sort_values(ascending=False, inplace=True)
hotspot04.head(5) # Top 5 hotspot in Toronto Cluster 4

Music Venue       0.333333
Ice Cream Shop    0.333333
Garden            0.333333
Dog Run           0.000000
Dessert Shop      0.000000
dtype: float64